# Data Preparation

## Import Modules

In [13]:
import os
import pandas as pd
import duckdb
import numpy as np

## Constants

In [19]:
# San Diego Country Zip Codes 
zipcodes = [
    91901, 91902, 91903, 91905, 91906, 91908, 91909, 91910, 91911, 91913,
    91914, 91915, 91916, 91917, 91919, 91921, 91931, 91932, 91934, 91935,
    91941, 91942, 91943, 91944, 91945, 91946, 91947, 91948, 91950, 91951,
    91962, 91963, 91964, 91965, 91976, 91977, 91978, 91979, 91980, 91987,
    92003, 92004, 92007, 92008, 92009, 92010, 92011, 92013, 92014, 92018,
    92019, 92020, 92021, 92022, 92023, 92024, 92025, 92026, 92027, 92028,
    92029, 92033, 92036, 92037, 92038, 92039, 92040, 92046, 92049, 92051,
    92052, 92054, 92055, 92056, 92057, 92058, 92059, 92060, 92061, 92062,
    92064, 92065, 92066, 92067, 92068, 92069, 92070, 92071, 92072, 92074,
    92075, 92078, 92079, 92082, 92083, 92084, 92085, 92086, 92088, 92090,
    92091, 92092, 92093, 92096, 92099, 92101, 92102, 92103, 92104, 92105,
    92106, 92107, 92108, 92109, 92110, 92111, 92112, 92113, 92114, 92115,
    92116, 92117, 92118, 92119, 92120, 92121, 92122, 92123, 92124, 92126,
    92127, 92128, 92129, 92130, 92131, 92132, 92133, 92134, 92135, 92136,
    92137, 92138, 92139, 92140, 92142, 92143, 92144, 92145, 92147, 92149,
    92150, 92152, 92153, 92154, 92155, 92158, 92159, 92160, 92161, 92162,
    92163, 92164, 92165, 92166, 92167, 92168, 92169, 92170, 92171, 92172,
    92173, 92174, 92175, 92176, 92177, 92178, 92179, 92180, 92182, 92184,
    92186, 92187, 92190, 92191, 92192, 92193, 92194, 92195, 92196, 92197,
    92198, 92199
]

GLOBAL_PLACES_DIR = 'data/global-places-poi-geometry-unl_01k8hy59vybc1vzqp3s5m6zhrg'
SPEND_PATTERNS_DIR = 'data/dewey-downloads/safegraph-spend-patterns-ca-only'

In [8]:
# Load the Global Places dataset
GLOBAL_PLACES_PATH = os.path.join(GLOBAL_PLACES_DIR, '*.parquet')
con = duckdb.connect()

# Filter for only San Diego County zip codes
df = con.execute(f"""
            SELECT
                PLACEKEY,
                SAFEGRAPH_PLACE_ID
                SAFEGRAPH_BRAND_IDS,
                LOCATION_NAME,
                BRANDS,
                TOP_CATEGORY,
                NAICS_CODE,
                LATITUDE,
                LONGITUDE,
                STREET_ADDRESS,
                CITY,
                POSTAL_CODE,
                REGION,
                CATEGORY_TAGS,
                WEBSITE
            FROM '{GLOBAL_PLACES_PATH}' 
            WHERE POSTAL_CODE IN ({','.join(map(str, zipcodes))})""").fetchdf()
df.head()

,PLACEKEY,SAFEGRAPH_BRAND_IDS,LOCATION_NAME,BRANDS,TOP_CATEGORY,NAICS_CODE,LATITUDE,LONGITUDE,STREET_ADDRESS,CITY,POSTAL_CODE,REGION,CATEGORY_TAGS,WEBSITE
0,zzy-222@5z5-pn4-28v,sg:51cf7faa5bfb39199e9b9316930b2afa,EvoNexus,None,Social Advocacy Organizations,813311.0,32.851034,-117.184413,5015 Shoreham Pl Ste 150,San Diego,92122.0,CA,None,None
1,zzy-222@5z5-pnk-vcq,sg:150b6c90c9c139f49e61586d03f04f64,USPS Collection Point,USPS Collection Point,Couriers and Express Delivery Services,492110.0,32.903090,-117.176567,10155 Mesa Rim Rd,San Diego,92121.0,CA,"Couriers,Mail Boxes,Packing Services,Packing S...",https://tools.usps.com/find-location.htm?locat...
2,zzy-222@5z5-pp7-x5z,sg:9b7d089dec524909860ae5028c795864,Ironclad Sportfishing,None,"Scenic and Sightseeing Transportation, Water",487210.0,32.763864,-117.236582,1677 Quivira Rd,San Diego,92109.0,CA,"Boat Charters,Boat Tours,Boats,Fishing,Tours",None
3,zzy-222@5z5-pp8-gff,sg:6e5e276c264e33ffa19aba80e0e88e48,Metropolitan Transit System West Point Loma Bl...,None,Urban Transit Systems,485113.0,32.752228,-117.232608,W Point Loma Blvd @ Temecula St,San Diego,92107.0,CA,"Bus Station,Buses",None
4,zzy-222@5z5-pvf-ndv,sg:3695084376c84fdc97d9f275a332542d,Rancho Penasquitos Off Leash Area,None,"Museums, Historical Sites, and Similar Institu...",712190.0,32.957520,-117.121708,Salmon River Rd @ Fairgrove Ln,San Diego,92129.0,CA,Dog Park,None


In [9]:
# export subset to parquet
df.to_parquet('data/san-diego-county-places.parquet')

In [10]:
df.columns

Index(['PLACEKEY', 'SAFEGRAPH_BRAND_IDS', 'LOCATION_NAME', 'BRANDS',
       'TOP_CATEGORY', 'NAICS_CODE', 'LATITUDE', 'LONGITUDE', 'STREET_ADDRESS',
       'CITY', 'POSTAL_CODE', 'REGION', 'CATEGORY_TAGS', 'WEBSITE'],
      dtype='object')

In [14]:
np.sort(df['TOP_CATEGORY'].unique())

array(['Accounting, Tax Preparation, Bookkeeping, and Payroll Services',
       'Activities Related to Credit Intermediation',
       'Activities Related to Real Estate',
       'Administration of Economic Programs',
       'Administration of Human Resource Programs',
       'Advertising, Public Relations, and Related Services',
       'Aerospace Product and Parts Manufacturing',
       'Agencies, Brokerages, and Other Insurance Related Activities',
       'Agents and Managers for Artists, Athletes, Entertainers, and Other Public Figures',
       'Agriculture, Forestry, Fishing and Hunting',
       'Ambulatory Health Care Services', 'Amusement Parks and Arcades',
       'Amusement, Gambling, and Recreation Industries',
       'Apparel Accessories and Other Apparel Manufacturing',
       'Apparel, Piece Goods, and Notions Merchant Wholesalers',
       'Architectural and Structural Metals Manufacturing',
       'Architectural, Engineering, and Related Services',
       'Arts, Entertainme

In [15]:
df['TOP_CATEGORY'].value_counts()

TOP_CATEGORY
Offices of Physicians                                                33218
Offices of Other Health Practitioners                                18807
Lessors of Real Estate                                               16774
Restaurants and Other Eating Places                                  16282
Personal Care Services                                               11929
                                                                     ...  
Hospitals                                                                1
Broadcasting (except Internet)                                           1
Beer, Wine, and Distilled Alcoholic Beverage Merchant Wholesalers        1
Hardware Manufacturing                                                   1
Commercial and Service Industry Machinery Manufacturing                  1
Name: count, Length: 270, dtype: int64

## Spend Patterns

In [31]:
SPEND_PATTERNS_PATH = os.path.join(SPEND_PATTERNS_DIR, '2025-07--data_01bffb7b-0106-c81a-0042-fa0703e58316_108_2_0.snappy.parquet')

spend_df = con.execute(f"""
            SELECT 
                *
            FROM '{SPEND_PATTERNS_PATH}'
            WHERE POSTAL_CODE IN ({','.join(map(str, zipcodes))}) AND
            BRANDS IS NOT NULL""").fetchdf()
spend_df.head()

,BRANDS,BUCKETED_CUSTOMER_FREQUENCY,BUCKETED_CUSTOMER_INCOMES,CITY,CUSTOMER_HOME_CITY,DAY_COUNTS,LOCATION_NAME,MEAN_SPEND_PER_CUSTOMER_BY_FREQUENCY,MEAN_SPEND_PER_CUSTOMER_BY_INCOME,MEDIAN_SPEND_PER_CUSTOMER,...,SPEND_DATE_RANGE_END,SPEND_DATE_RANGE_START,SPEND_PCT_CHANGE_VS_PREV_MONTH,SPEND_PCT_CHANGE_VS_PREV_YEAR,SPEND_PER_TRANSACTION_BY_DAY,SPEND_PER_TRANSACTION_PERCENTILES,STREET_ADDRESS,SUB_CATEGORY,TOP_CATEGORY,TRANSACTION_INTERMEDIARY
0,Jack in the Box,"{""1"":4,""2"":0,""3"":0,""4"":0,""5-10"":0,"">10"":0}","{""60-75k"":2,""75-100k"":2,"">150k"":3}",National City,"{""key_value"":[{""key"":"", ND"",""value"":2},{""key"":...","{""Friday"":4,""Monday"":4,""Saturday"":4,""Sunday"":4...",Jack in the Box,"{""1"":22.04}","{""60-75k"":5.96,""75-100k"":19.87,"">150k"":31.16}",25.52,...,2025-08-01,2025-07-01,NaN,NaN,"[null,null,31.16,null,null,null,null,5.96,null...","{""25"":9.44,""75"":31.16}",700 Roosevelt Ave,Limited-Service Restaurants,Restaurants and Other Eating Places,"{""key_value"":[{""key"":""No intermediary"",""value""..."
1,Walmart,"{""1"":1378,""2"":288,""3"":73,""4"":34,""5-10"":48,"">10...","{""100-150k"":212,""25-45k"":340,""45-60k"":239,""60-...",Poway,"{""key_value"":[{""key"":""Richland, WA"",""value"":2}...","{""Friday"":4,""Monday"":4,""Saturday"":4,""Sunday"":4...",Walmart Supercenter,"{""1"":45.64,""2"":91.39,""3"":210.87,""4"":164.88,""5-...","{""100-150k"":63.73,""25-45k"":48.44,""45-60k"":51.1...",33.93,...,2025-08-01,2025-07-01,36.0,167.0,"[28.69,39.41,68.28,26.95,18.77,49.75,30.63,28....","{""25"":12,""75"":46.7}",13425 Community Rd,Warehouse Clubs and Supercenters,"General Merchandise Stores, including Warehous...","{""key_value"":[{""key"":""No intermediary"",""value""..."
2,Carl's Jr.,"{""1"":18,""2"":1,""3"":0,""4"":0,""5-10"":0,"">10"":0}","{""100-150k"":2,""25-45k"":2,""45-60k"":2,""60-75k"":2...",La Mesa,"{""key_value"":[{""key"":""Encinitas, CA"",""value"":2...","{""Friday"":4,""Monday"":4,""Saturday"":4,""Sunday"":4...",Carl's Jr.,"{""1"":18.13,""2"":26.11}","{""100-150k"":24.16,""25-45k"":11.38,""45-60k"":26.3...",16.14,...,2025-08-01,2025-07-01,55.0,7.0,"[24.71,null,null,16.14,null,null,24.16,null,7....","{""25"":12.56,""75"":24.57}",8110 Parkway Dr,Limited-Service Restaurants,Restaurants and Other Eating Places,"{""key_value"":[{""key"":""No intermediary"",""value""..."
3,24 Hour Fitness,"{""1"":2,""2"":1,""3"":1,""4"":0,""5-10"":0,"">10"":0}","{""25-45k"":2,"">150k"":2}",San Diego,"{""key_value"":[{""key"":""Lakeside, CA"",""value"":2}...","{""Friday"":4,""Monday"":4,""Saturday"":4,""Sunday"":4...",24 Hour Fitness,"{""1"":4.78,""2"":40,""3"":21.25}","{""25-45k"":10.27,"">150k"":40}",13.37,...,2025-08-01,2025-07-01,NaN,NaN,"[null,null,null,null,20,5.48,null,null,null,8....","{""25"":4.29,""75"":20}",5885 Rancho Mission Rd,Fitness and Recreational Sports Centers,Other Amusement and Recreation Industries,"{""key_value"":[{""key"":""No intermediary"",""value""..."
4,Great Clips,"{""1"":10,""2"":3,""3"":0,""4"":0,""5-10"":0,"">10"":0}","{""100-150k"":2,""25-45k"":2,""45-60k"":2,""60-75k"":2...",Oceanside,"{""key_value"":[{""key"":""Oceanside, CA"",""value"":9...","{""Friday"":4,""Monday"":4,""Saturday"":4,""Sunday"":4...",Great Clips,"{""1"":33.6,""2"":51.67}","{""100-150k"":46,""25-45k"":31.5,""45-60k"":33.66,""6...",35.00,...,2025-08-01,2025-07-01,-4.0,-6.0,"[null,null,null,null,26,null,39,24.5,28,30,nul...","{""25"":25.99,""75"":34}",3825 Mission Ave,Beauty Salons,Personal Care Services,"{""key_value"":[{""key"":""No intermediary"",""value""..."


In [32]:
spend_df.to_parquet('data/san-diego-county-spend-patterns.parquet')